In [2]:
import pandas as pd
import time
import logging
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from model_config import load_gemini2
# Khởi tạo logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
from retrieval import hybrid_retriever, bm25_retriever
# Khởi tạo LangChain Chat Model
llm = load_gemini2()  # Giả sử bạn đã có hàm này để tải mô hình
#db = hybrid_retriever("base")
db = bm25_retriever("raptor")
# Prompt để sinh câu trả lời
template = """
Bạn là chuyên gia trong ngành giáo dục. Nhiệm vụ của bạn là tạo ra các câu trả lời cho phần Câu hỏi bên dưới. 
Hãy tạo câu trả lời phù hợp cho câu hỏi được cung cấp bên dưới. 
## Quan trọng: Các câu trả lời phải được viết bằng tiếng Việt.
Chỉ tạo câu trả lời dựa trên thông tin trong văn bản được cung cấp.

##Dưới đây là những văn bản liên quan:
{context}
##Đây là câu hỏi: 
{question}
##Câu trả lời của bạn là:

Nếu không có đủ thông tin để tạo câu trả lời, trả lời: "Không thể trả lời."
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

# Đọc file Excel đầu vào
input_file = r"D:\DATN\QA_System\eval\testset\output_dataset_out.xlsx"
output_file = r"D:\DATN\QA_System\eval\testset\output_dataset_out.xlsx"
data = pd.read_excel(input_file)

# # Lọc bỏ các dòng có giá trị [] trong cột `filtered_questions`
# data = data[data['filtered_questions'] != '[]']

# Hàm sinh câu trả lời với retry
def generate_answers_with_retry(context, question, retries=5, backoff_factor=2):
    attempt = 0
    while attempt < retries:
        try:
            logging.info(f"Đang tạo câu trả lời cho câu hỏi:")
            result = chain.invoke({"context": context, "question": question})  # Tạo câu trả lời
            return result.strip()
        except Exception as e:
            if "ResourceExhausted" in str(e):
                attempt += 1
                wait_time = backoff_factor ** attempt
                logging.warning(f"Lỗi khi tạo câu trả lời: {e}. Thử lại sau {wait_time} giây...")
                time.sleep(wait_time)
            else:
                logging.error(f"Lỗi không xác định: {e}")
                return "Không thể tạo câu trả lời."
    return "Không thể tạo câu trả lời sau nhiều lần thử lại."

# Sinh câu trả lời cho từng dòng
answers_list = []
for index, row in data.iterrows():

    question = row['Question']
    logging.info(f"Đang xử lý dòng {index+1}/{len(data)}...")
    relevant_docs = db.invoke(question)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    # Sinh câu trả lời với retry
    answer = generate_answers_with_retry(context, question)
    answers_list.append(answer)
    
    # Thêm sleep giữa các lần gọi API để không vượt quá giới hạn tốc độ
    time.sleep(4.1)

data['Answerbm25raptor'] = answers_list

# Lưu file kết quả
data.to_excel(output_file, index=False, engine="openpyxl")
logging.info(f"File đã được cập nhật và lưu tại {output_file}")


2025-01-05 21:14:13,086 - INFO - GET https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443/ [status:200 duration:0.704s]
2025-01-05 21:14:13,107 - INFO - Đang xử lý dòng 1/90...
2025-01-05 21:14:13,289 - INFO - POST https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443/raptor/_search [status:200 duration:0.181s]
2025-01-05 21:14:13,292 - INFO - Đang tạo câu trả lời cho câu hỏi:
2025-01-05 21:14:18,957 - INFO - Đang xử lý dòng 2/90...
2025-01-05 21:14:19,134 - INFO - POST https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443/raptor/_search [status:200 duration:0.174s]
2025-01-05 21:14:19,134 - INFO - Đang tạo câu trả lời cho câu hỏi:
2025-01-05 21:14:24,117 - INFO - Đang xử lý dòng 3/90...
2025-01-05 21:14:24,303 - INFO - POST https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443/raptor/_search [status:200 duration:0.185s]
2025-01-05 21:14:24,306 - INFO - Đang tạo câu trả lời cho câu 